In [1]:
from typing import List

def color_correction_procedure(adj : List[List[int]], colors : List[int]):
    # colors = list(map(lambda x: int(round(x)), colors))

    old_colors = len(set(colors))

    colors_index = dict()

    for index, color in enumerate(colors):
        if color not in colors_index:
            n = len(colors_index)
            colors_index[color] = n
            colors[index] = n
        else:
            colors[index] = colors_index[color]
    
    n = len(adj)
    colors_count = len(set(colors))

    for i in range(n):
        for j in range(i + 1, n):
            if adj[i][j] == 0:
                continue
        
            if colors[i] != colors[j]:
                continue

            neighbours = [index for index, elem in enumerate(adj[i]) if elem == 1]

            nei_colors = set(map(lambda nei : colors[nei], neighbours))
            
            available = set(colors).difference(nei_colors)

            if len(available) > 0:
                new_color = list(available)[0]

                colors[i] = new_color
                continue

            neighbours = [index for index, elem in enumerate(adj[j]) if elem == 1]
            nei_colors = set(map(lambda nei : colors[nei], neighbours))
            
            available = set(colors).difference(nei_colors)

            if len(available) > 0:
                new_color = list(available)[0]

                colors[j] = new_color
                continue
                
            # none of the above, we need to create a new color

            new_color = colors_count
            colors_count += 1

            colors[i] = new_color

    return old_colors, len(set(colors)), colors

In [2]:
import numpy as np
from gen_data.data_generation import generate_random_data, generate_clique_data

sizes = list(map(int, np.linspace(10, 100, 5)))
sparsities = list(np.logspace(-2, -1, 4))

NUM_GRAPHS = 100

random_data = generate_random_data(NUM_GRAPHS, sizes, sparsity_levels=sparsities)
clique_data = generate_clique_data(NUM_GRAPHS, sizes, sparsity_levels=sparsities)

In [3]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]

        sample = torch.tensor(sample, dtype=torch.float32)

        if self.transform:
            sample = self.transform(sample)

        return sample

In [4]:
# prepare the data for various forms of usage: raw graph, Dataset with adjacency matrix, Dataset with linearized matrix
from gen_data.data_generation import compressed_to_adj, graph_to_compressed
from itertools import chain

random_adjacency = list(map(lambda g: compressed_to_adj(graph_to_compressed(g)), random_data))
clique_adjacency = list(map(lambda g: compressed_to_adj(graph_to_compressed(g)), clique_data))

random_dataloader = torch.utils.data.DataLoader(dataset=CustomDataset(data=random_adjacency), batch_size=1, shuffle=False)
clique_dataloader = torch.utils.data.DataLoader(dataset=CustomDataset(data=clique_adjacency), batch_size=1, shuffle=False)

random_linearized = list(map(lambda mat: list(chain(mat)), random_adjacency))
random_linearized = torch.utils.data.DataLoader(dataset=CustomDataset(data=random_linearized), batch_size=1, shuffle=False)

clique_linearized = list(map(lambda mat: list(chain(mat)), clique_adjacency))
clique_linearized = torch.utils.data.DataLoader(dataset=CustomDataset(data=clique_linearized), batch_size=1, shuffle=False)

In [5]:
results = {
    "bkt_colors" : [],
    "bkt_time" : [],
    "sat_colors" : [],
    "sat_time" : [],
    "heuristic_colors" : [],
    "heuristic_time" : [],
    "base_initial_colors" : [],
    "base_colors" : [],
    "base_time" : [],
    "proposed_initial_colors" : [],
    "proposed_colors" : [],
    "proposed_time" : [],
    "shuffle_initial_colors" : [],
    "shuffle_colors" : [],
    "shuffle_time" : [],
    "size": [],
    "sparsity": []
}

In [6]:
from comparison.bkt import solve_bkt
import time
import signal

TIMEOUT = 10

def handler(signum, frame):
    raise Exception("Timeout for this graph !")

signal.signal(signal.SIGALRM, handler)

for index, graph in enumerate(random_data):
    count = -1
    interval = TIMEOUT

    signal.alarm(TIMEOUT)
    try:
        start = time.time()
        count, colors = solve_bkt(graph)
        end = time.time()

        signal.alarm(0)

        print(f"Graph {index} is finished !")
        interval = end - start
    except Exception as e:
        print(f"Graph {index} timed out !")
    
    results['bkt_colors'].append(count)
    results['bkt_time'].append(interval)

Graph 0 is finished !
Graph 1 is finished !
Graph 2 is finished !
Graph 3 is finished !
Graph 4 is finished !
Graph 5 is finished !
Graph 6 is finished !
Graph 7 is finished !
Graph 8 is finished !
Graph 9 is finished !
Graph 10 is finished !
Graph 11 is finished !
Graph 12 is finished !
Graph 13 is finished !
Graph 14 is finished !
Graph 15 is finished !
Graph 16 is finished !
Graph 17 is finished !
Graph 18 is finished !
Graph 19 is finished !
Graph 20 is finished !
Graph 21 is finished !
Graph 22 is finished !
Graph 23 is finished !
Graph 24 is finished !
Graph 25 is finished !
Graph 26 is finished !
Graph 27 is finished !
Graph 28 is finished !
Graph 29 is finished !
Graph 30 is finished !
Graph 31 is finished !
Graph 32 is finished !
Graph 33 is finished !
Graph 34 is finished !
Graph 35 is finished !
Graph 36 is finished !
Graph 37 is finished !
Graph 38 is finished !
Graph 39 is finished !
Graph 40 timed out !
Graph 41 timed out !
Graph 42 timed out !
Graph 43 timed out !
Graph 

In [7]:
from comparison.sat import solve_sat
import time

results['sat_colors'] = []
results['sat_time'] = []

for index, graph in enumerate(random_data):
    start = time.time()
    count, colors = solve_sat(graph)
    end = time.time()

    print(f"Graph {index} is finished !")
    results['sat_colors'].append(count)
    results['sat_time'].append(end - start)

Graph 0 is finished !
Graph 1 is finished !
Graph 2 is finished !
Graph 3 is finished !
Graph 4 is finished !
Graph 5 is finished !
Graph 6 is finished !
Graph 7 is finished !
Graph 8 is finished !
Graph 9 is finished !
Graph 10 is finished !
Graph 11 is finished !
Graph 12 is finished !
Graph 13 is finished !
Graph 14 is finished !
Graph 15 is finished !
Graph 16 is finished !
Graph 17 is finished !
Graph 18 is finished !
Graph 19 is finished !
Graph 20 is finished !
Graph 21 is finished !
Graph 22 is finished !
Graph 23 is finished !
Graph 24 is finished !
Graph 25 is finished !
Graph 26 is finished !
Graph 27 is finished !
Graph 28 is finished !
Graph 29 is finished !
Graph 30 is finished !
Graph 31 is finished !
Graph 32 is finished !
Graph 33 is finished !
Graph 34 is finished !
Graph 35 is finished !
Graph 36 is finished !
Graph 37 is finished !
Graph 38 is finished !
Graph 39 is finished !
Graph 40 is finished !
Graph 41 is finished !
Graph 42 is finished !
Graph 43 is finished 

In [8]:
from comparison.heuristic import solve_heuristic
import time

for index, graph in enumerate(random_data):
    start = time.time()
    count, colors = solve_heuristic(graph)
    end = time.time()

    print(f"Graph {index} is finished !")
    results['heuristic_colors'].append(count)
    results['heuristic_time'].append(end - start)

Graph 0 is finished !
Graph 1 is finished !
Graph 2 is finished !
Graph 3 is finished !
Graph 4 is finished !
Graph 5 is finished !
Graph 6 is finished !
Graph 7 is finished !
Graph 8 is finished !
Graph 9 is finished !
Graph 10 is finished !
Graph 11 is finished !
Graph 12 is finished !
Graph 13 is finished !
Graph 14 is finished !
Graph 15 is finished !
Graph 16 is finished !
Graph 17 is finished !
Graph 18 is finished !
Graph 19 is finished !
Graph 20 is finished !
Graph 21 is finished !
Graph 22 is finished !
Graph 23 is finished !
Graph 24 is finished !
Graph 25 is finished !
Graph 26 is finished !
Graph 27 is finished !
Graph 28 is finished !
Graph 29 is finished !
Graph 30 is finished !
Graph 31 is finished !
Graph 32 is finished !
Graph 33 is finished !
Graph 34 is finished !
Graph 35 is finished !
Graph 36 is finished !
Graph 37 is finished !
Graph 38 is finished !
Graph 39 is finished !
Graph 40 is finished !
Graph 41 is finished !
Graph 42 is finished !
Graph 43 is finished 

In [9]:
print(results['heuristic_colors'])
print(results['sat_colors'])

[2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 4, 5, 4, 5, 5, 5, 6, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 6, 5, 6, 7, 6, 6, 6, 6, 7, 8, 7, 7, 7, 6, 7, 7, 7, 8, 7, 8, 7, 7, 7, 7, 7, 7, 7, 7]
[2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 5, 5, 5, 4, 4, 5, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [12]:
import torch
from torch import nn
from torch.nn import LSTM
from torch.autograd import Variable

class ColoringNN(nn.LSTM):
    def __init__(self, in_size : int, out_size : int,
                  hidden_size : int, num_layers : int, num_nodes : int):
        super(ColoringNN, self).__init__(input_size=in_size, hidden_size=hidden_size)

        self.in_size = in_size
        self.out_size = out_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_nodes = num_nodes

        self.cells = nn.ModuleList([
            LSTM(
                input_size = in_size,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=True
            )
            for _ in range(num_nodes)
        ])

        self.linear = nn.Linear(in_features=hidden_size, out_features=1)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        outputs = []

        for i, lstm in enumerate(self.cells):
            out, _ = lstm(inputs[:, [i], :])

            out = self.linear(out)
            out = torch.flatten(input=out, start_dim=0)
            out = self.relu(out)

            outputs.append(out.unsqueeze(-1))

        res = torch.cat(outputs, dim=1)
        return res

model = torch.load('model_base2.pth')

model.eval()

res = []

results['base_time'] = []
results['base_initial_colors'] = []
results['base_colors'] = []

for index, inputs in enumerate(random_dataloader):
    inputs = Variable(inputs)

    with torch.no_grad():
        start = time.time()
        out = model(inputs)
        end = time.time()
        res.append(out)
        results['base_time'].append(end - start)

res = list(map(lambda t: torch.flatten(t, start_dim=0), res))
res = list(map(lambda t: list(t.detach().numpy()), res))
res = list(map(lambda cols: list(map(lambda col: int(round(col)), cols)), res))
results['base_initial_colors'] = []

for index, adj in enumerate(random_adjacency):
    old, new, _ = color_correction_procedure(adj, res[index])
    results['base_colors'].append(new)
    results['base_initial_colors'].append(old)

In [13]:
print(results['base_colors'])
print(results['base_initial_colors'])
print(results['sat_colors'])
print(results['heuristic_colors'])

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 3, 2, 2, 3, 3, 4, 3, 3, 4, 4, 5, 5, 5, 5, 6, 6, 5, 5, 6, 6, 5, 5, 6, 5, 6, 5, 6, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 5, 5, 6, 6, 6, 6, 5, 6, 6, 6, 5, 5, 6, 6, 5, 5, 5, 5, 7, 5, 6, 6, 5, 6, 5, 6, 6, 5, 6, 5, 5, 6]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 2, 3, 3, 3, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3]
[2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 5, 5, 5, 4, 4, 5, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
[2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 

In [14]:
import torch
from torch import nn
from torch.nn import LSTM
from torch.autograd import Variable

class ColoringNN(nn.LSTM):
    def __init__(self, in_size : int, out_size : int,
                  hidden_size : int, num_layers : int, num_nodes : int):
        super(ColoringNN, self).__init__(input_size=in_size, hidden_size=hidden_size)

        self.in_size = in_size
        self.out_size = out_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_nodes = num_nodes

        self.lstm = LSTM(
                input_size = in_size,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=True
            )

        # self.linears = nn.ModuleList(
        #     [nn.Linear(in_features=hidden_size, out_features=1) for _ in range(num_nodes)]
        # )

        self.linear = nn.Linear(in_features=hidden_size, out_features=100)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        outputs, _ = self.lstm(inputs)
        outputs = self.linear(outputs)
        outputs = self.relu(outputs)
        return outputs

model = torch.load('model_original2.pth')

model.eval()

res = []

results['proposed_time'] = []
results['proposed_colors'] = []
results['proposed_initial_colors'] = []

for index, inputs in enumerate(random_linearized):
    inputs = Variable(inputs)

    with torch.no_grad():
        start = time.time()
        out = model(inputs)
        end = time.time()
        res.append(out)
        results['proposed_time'].append(end - start)

# print(res)
res = list(map(lambda t: torch.flatten(t, start_dim=0), res))
res = list(map(lambda t: list(t.detach().numpy()), res))
res = list(map(lambda cols: list(map(lambda col: int(round(col)), cols)), res))

for index, adj in enumerate(random_adjacency):
    old, new, _ = color_correction_procedure(adj, res[index])
    results['proposed_colors'].append(new)
    results['proposed_initial_colors'].append(old)

In [16]:
print(results['base_colors'])
print(results['base_initial_colors'])
print(results['proposed_colors'])
print(results['proposed_initial_colors'])
print(results['sat_colors'])
print(results['heuristic_colors'])

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 3, 2, 2, 3, 3, 4, 3, 3, 4, 4, 5, 5, 5, 5, 6, 6, 5, 5, 6, 6, 5, 5, 6, 5, 6, 5, 6, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 5, 5, 6, 6, 6, 6, 5, 6, 6, 6, 5, 5, 6, 6, 5, 5, 5, 5, 7, 5, 6, 6, 5, 6, 5, 6, 6, 5, 6, 5, 5, 6]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 2, 3, 3, 3, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3]
[2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 3, 3, 2, 3, 3, 4, 3, 3, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 6, 6, 5, 5, 5, 5, 6, 6, 5, 5, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 5, 6, 5, 6, 6, 5, 6, 5, 5, 6, 5, 6]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [17]:
import torch
from torch import nn
from torch.nn import LSTM
from torch.autograd import Variable

class ColoringNN(nn.LSTM):
    def __init__(self, in_size : int, out_size : int,
                  hidden_size : int, num_layers : int, num_nodes : int):
        super(ColoringNN, self).__init__(input_size=in_size, hidden_size=hidden_size)

        self.in_size = in_size
        self.out_size = out_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_nodes = num_nodes

        self.cells = nn.ModuleList([
            LSTM(
                input_size = in_size,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=True
            )
            for _ in range(num_nodes)
        ])

        self.linear = nn.Linear(in_features=hidden_size, out_features=1)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        outputs = []

        for i, lstm in enumerate(self.cells):
            out, _ = lstm(inputs[:, [i], :])

            out = self.linear(out)
            out = torch.flatten(input=out, start_dim=0)
            out = self.relu(out)

            outputs.append(out.unsqueeze(-1))

        res = torch.cat(outputs, dim=1)
        return res

model = torch.load('model_base_shuffle.pth')

model.eval()

res = []

results['shuffle_time'] = []
results['shuffle_initial_colors'] = []
results['shuffle_colors'] = []

for index, inputs in enumerate(random_dataloader):
    inputs = Variable(inputs)

    with torch.no_grad():
        start = time.time()
        out = model(inputs)
        end = time.time()
        res.append(out)
        results['shuffle_time'].append(end - start)

res = list(map(lambda t: torch.flatten(t, start_dim=0), res))
res = list(map(lambda t: list(t.detach().numpy()), res))
res = list(map(lambda cols: list(map(lambda col: int(round(col)), cols)), res))
results['shuffle_initial_colors'] = []

for index, adj in enumerate(random_adjacency):
    old, new, _ = color_correction_procedure(adj, res[index])
    results['shuffle_colors'].append(new)
    results['shuffle_initial_colors'].append(old)

In [18]:
print(results['base_colors'])
# print(results['base_initial_colors'])
print(results['proposed_colors'])
print(results['shuffle_colors'])
# print(results['proposed_initial_colors'])
print(results['sat_colors'])
print(results['heuristic_colors'])

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 3, 2, 2, 3, 3, 4, 3, 3, 4, 4, 5, 5, 5, 5, 6, 6, 5, 5, 6, 6, 5, 5, 6, 5, 6, 5, 6, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 5, 5, 6, 6, 6, 6, 5, 6, 6, 6, 5, 5, 6, 6, 5, 5, 5, 5, 7, 5, 6, 6, 5, 6, 5, 6, 6, 5, 6, 5, 5, 6]
[2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 3, 3, 2, 3, 3, 4, 3, 3, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 6, 6, 5, 5, 5, 5, 6, 6, 5, 5, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 5, 6, 5, 6, 6, 5, 6, 5, 5, 6, 5, 6]
[2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 3, 3, 2, 3, 3, 4, 3, 3, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 6, 6, 5, 5, 5, 5, 6, 6, 5, 5, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 5, 6, 5, 6, 6, 5, 6, 5, 5, 6, 5, 6]
[2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 

In [33]:
import pandas as pd

results['sizes'] = list(map(lambda g: g.vcount(), random_data))
results['sparsities'] = list(map(lambda g : g.ecount() * 2 / (g.vcount() ** 2), random_data))
results_df = pd.DataFrame(results)

results_df.to_csv('comparison1.csv', index=False)

In [20]:
comparison = {
    "random_colors_random" : [],
    "random_colors_clique" : [],
    "clique_colors_random" : [],
    "clique_colors_clique" : [],
    "exact_random": results['sat_colors'],
    "exact_clique": []
}

In [22]:
from comparison.sat import solve_sat
import time

comparison['exact_clique'] = []

for index, graph in enumerate(clique_data):
    count, colors = solve_sat(graph)

    print(f"Graph {index} is finished !")
    comparison['exact_clique'].append(count)

Graph 0 is finished !
Graph 1 is finished !
Graph 2 is finished !
Graph 3 is finished !
Graph 4 is finished !
Graph 5 is finished !
Graph 6 is finished !
Graph 7 is finished !
Graph 8 is finished !
Graph 9 is finished !
Graph 10 is finished !
Graph 11 is finished !
Graph 12 is finished !
Graph 13 is finished !
Graph 14 is finished !
Graph 15 is finished !
Graph 16 is finished !
Graph 17 is finished !
Graph 18 is finished !
Graph 19 is finished !
Graph 20 is finished !
Graph 21 is finished !
Graph 22 is finished !
Graph 23 is finished !
Graph 24 is finished !
Graph 25 is finished !
Graph 26 is finished !
Graph 27 is finished !
Graph 28 is finished !
Graph 29 is finished !
Graph 30 is finished !
Graph 31 is finished !
Graph 32 is finished !
Graph 33 is finished !
Graph 34 is finished !
Graph 35 is finished !
Graph 36 is finished !
Graph 37 is finished !
Graph 38 is finished !
Graph 39 is finished !
Graph 40 is finished !
Graph 41 is finished !
Graph 42 is finished !
Graph 43 is finished 

In [23]:
import torch
from torch import nn
from torch.nn import LSTM
from torch.autograd import Variable

class ColoringNN(nn.LSTM):
    def __init__(self, in_size : int, out_size : int,
                  hidden_size : int, num_layers : int, num_nodes : int):
        super(ColoringNN, self).__init__(input_size=in_size, hidden_size=hidden_size)

        self.in_size = in_size
        self.out_size = out_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_nodes = num_nodes

        self.cells = nn.ModuleList([
            LSTM(
                input_size = in_size,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=True
            )
            for _ in range(num_nodes)
        ])

        self.linear = nn.Linear(in_features=hidden_size, out_features=1)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        outputs = []

        for i, lstm in enumerate(self.cells):
            out, _ = lstm(inputs[:, [i], :])

            out = self.linear(out)
            out = torch.flatten(input=out, start_dim=0)
            out = self.relu(out)

            outputs.append(out.unsqueeze(-1))

        res = torch.cat(outputs, dim=1)
        return res

model = torch.load('model_base.pth')

model.eval()

res = []

for index, inputs in enumerate(random_dataloader):
    inputs = Variable(inputs)

    with torch.no_grad():
        out = model(inputs)
        res.append(out)

res = list(map(lambda t: torch.flatten(t, start_dim=0), res))
res = list(map(lambda t: list(t.detach().numpy()), res))
res = list(map(lambda cols: list(map(lambda col: int(round(col)), cols)), res))
comparison['random_colors_random'] = []

for index, adj in enumerate(random_adjacency):
    old, new, _ = color_correction_procedure(adj, res[index])
    comparison['random_colors_random'].append(new)

# clique
res = []

for index, inputs in enumerate(clique_dataloader):
    inputs = Variable(inputs)

    with torch.no_grad():
        out = model(inputs)
        res.append(out)

res = list(map(lambda t: torch.flatten(t, start_dim=0), res))
res = list(map(lambda t: list(t.detach().numpy()), res))
res = list(map(lambda cols: list(map(lambda col: int(round(col)), cols)), res))
comparison['random_colors_clique'] = []

for index, adj in enumerate(clique_adjacency):
    old, new, _ = color_correction_procedure(adj, res[index])
    comparison['random_colors_clique'].append(new)

In [24]:
import torch
from torch import nn
from torch.nn import LSTM
from torch.autograd import Variable

class ColoringNN(nn.LSTM):
    def __init__(self, in_size : int, out_size : int,
                  hidden_size : int, num_layers : int, num_nodes : int):
        super(ColoringNN, self).__init__(input_size=in_size, hidden_size=hidden_size)

        self.in_size = in_size
        self.out_size = out_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_nodes = num_nodes

        self.cells = nn.ModuleList([
            LSTM(
                input_size = in_size,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=True
            )
            for _ in range(num_nodes)
        ])

        self.linear = nn.Linear(in_features=hidden_size, out_features=1)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        outputs = []

        for i, lstm in enumerate(self.cells):
            out, _ = lstm(inputs[:, [i], :])

            out = self.linear(out)
            out = torch.flatten(input=out, start_dim=0)
            out = self.relu(out)

            outputs.append(out.unsqueeze(-1))

        res = torch.cat(outputs, dim=1)
        return res

model = torch.load('model_base_clique.pth')

model.eval()

res = []

for index, inputs in enumerate(random_dataloader):
    inputs = Variable(inputs)

    with torch.no_grad():
        out = model(inputs)
        res.append(out)

res = list(map(lambda t: torch.flatten(t, start_dim=0), res))
res = list(map(lambda t: list(t.detach().numpy()), res))
res = list(map(lambda cols: list(map(lambda col: int(round(col)), cols)), res))
comparison['clique_colors_random'] = []

for index, adj in enumerate(random_adjacency):
    old, new, _ = color_correction_procedure(adj, res[index])
    comparison['clique_colors_random'].append(new)

# clique
res = []

for index, inputs in enumerate(clique_dataloader):
    inputs = Variable(inputs)

    with torch.no_grad():
        out = model(inputs)
        res.append(out)

res = list(map(lambda t: torch.flatten(t, start_dim=0), res))
res = list(map(lambda t: list(t.detach().numpy()), res))
res = list(map(lambda cols: list(map(lambda col: int(round(col)), cols)), res))
comparison['clique_colors_clique'] = []

for index, adj in enumerate(clique_adjacency):
    old, new, _ = color_correction_procedure(adj, res[index])
    comparison['clique_colors_clique'].append(new)

In [34]:
comparison_df = pd.DataFrame(comparison)

comparison_df['random_sizes'] = list(map(lambda g: g.vcount(), random_data))
comparison_df['clique_sizes'] = list(map(lambda g: g.vcount(), clique_data))

comparison_df.to_csv('comparison2.csv', index=False)

In [29]:
comparison_df

,random_colors_random,random_colors_clique,clique_colors_random,clique_colors_clique,exact_random,exact_clique
0,4,4,2,1,2,1
1,4,4,2,1,2,1
2,4,4,1,1,1,1
3,4,4,2,1,2,1
4,4,4,1,1,1,1
...,...,...,...,...,...,...
95,6,7,5,6,5,6
96,6,6,6,6,5,6
97,5,7,5,7,5,6
98,5,7,5,7,5,6


In [ ]:
# TODO: parse the logs from model_original.log and plot the loss curve